# Underwater Semantic Segmentation (SUIM) – Assignment 2 Proposal 6
Custom UNet-ResAttn + baselines SUIM-Net and DeepLabV3.


## 1. Setup & Imports

In [ ]:
import os, sys
sys.path.append('..')
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from datasets.suim_dataset import SUIMDataset, CLASS_NAMES
from datasets.augmentations import train_transforms, val_transforms
from models.unet_resattn import UNetResAttn
from models.suimnet import SUIMNet
from models.deeplab_resnet import get_deeplabv3
from training.loss import DiceCELoss
from training.train import train_one_epoch, validate
from training.eval import evaluate_loader
from training.utils import count_parameters, save_checkpoint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


## 2. Create / Check Splits

In [ ]:
from data.utils.download_data import create_splits
if not os.path.exists('data/splits/train.txt'):
    create_splits('data/images', 'data/splits')
print('Splits OK')


## 3. Dataloaders

In [ ]:
train_ds = SUIMDataset('data/splits/train.txt', transform=train_transforms)
val_ds   = SUIMDataset('data/splits/val.txt', transform=val_transforms)
test_ds  = SUIMDataset('data/splits/test.txt', transform=val_transforms)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=2)


## 4. Train Custom UNet-ResAttn

In [ ]:
model = UNetResAttn().to(device)
criterion = DiceCELoss(dice_weight=0.5)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
print('Params:', count_parameters(model)/1e6, 'M')

best_iou = 0
history = {'train_loss':[], 'val_loss':[], 'train_iou':[], 'val_iou':[]}
for epoch in range(40):
    tr_loss, tr_iou = train_one_epoch(model, train_loader, optimizer, criterion, device)
    va_loss, va_iou = validate(model, val_loader, criterion, device)
    history['train_loss'].append(tr_loss)
    history['val_loss'].append(va_loss)
    history['train_iou'].append(tr_iou)
    history['val_iou'].append(va_iou)
    if va_iou > best_iou:
        best_iou = va_iou
        save_checkpoint(model, optimizer, epoch, best_iou, 'results/checkpoints/unet_resattn_best.pth')
    print(f'Epoch {epoch:02d} | tr IoU={tr_iou:.3f} va IoU={va_iou:.3f} tr loss={tr_loss:.3f} va loss={va_loss:.3f}')


## 5. Evaluate on Test Set (Custom Model)

In [ ]:
test_miou, test_per_class = evaluate_loader(model, test_loader, device)
print('Test mIoU:', test_miou)
for name, v in zip(CLASS_NAMES, test_per_class):
    print(name, v)


## 6. SUIM-Net baseline (train similarly)

In [ ]:
suim_model = SUIMNet().to(device)
print('Params:', count_parameters(suim_model)/1e6, 'M')
# Train with same loop if you want fair comparison.


## 7. DeepLabV3 baseline

In [ ]:
deeplab = get_deeplabv3(num_classes=8, pretrained=True).to(device)
print('Params:', count_parameters(deeplab)/1e6, 'M')
# deeplab(x) returns {'out': logits}
